In [1]:
import torch
import torchvision
import torch.nn as nn
from torch.optim import lr_scheduler
import numpy as np
from torchvision import datasets, transforms, models
import matplotlib.pyplot as plt
import time
import os
import copy


In [2]:
device = torch.device('cuda' if torch.cuda.is_available else 'cpu')
device

device(type='cuda')

In [3]:
mean = np.array([0.485,0.456,0.406])
std = np.array([0.229, 0.224, 0.225])

In [4]:
# data transfromations

data_transforms = {
    'train': transforms.Compose([
        transforms.RandomResizedCrop(224),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize(mean,std)
    ]),
    'val': transforms.Compose([
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize(mean,std)

    ])

}

In [5]:
from google.colab import drive

drive.mount('/content/drive')


Mounted at /content/drive


In [6]:
data_dir= '/content/drive/MyDrive/data/hymenoptera_data'
sets = ['train','val']
image_datasets = {x: datasets.ImageFolder(os.path.join(data_dir, x),
                                          data_transforms[x]) for x in ['train','val']}

In [7]:
dataloaders = {x: torch.utils.data.DataLoader(image_datasets[x], batch_size=4, shuffle=True, num_workers=0)
 for x in ['train','val']}

In [8]:
dataset_sizes = {x: len(image_datasets[x]) for x in ['train','val']}
dataset_sizes

{'train': 246, 'val': 153}

In [9]:
class_names = image_datasets['train'].classes
class_names

['ants', 'bees']

In [21]:
def train_model(model,criterion, optimizer, scheduler, num_epochs=25):
  since = time.time()
  best_model_wts = copy.deepcopy(model.state_dict())
  best_acc = 0.0

  for epoch in range(num_epochs):
    print(f'epoch {epoch}/{num_epochs-1}')
    print('-' * 10)

    # each epoch has a training and validation phase
    for phase in ['train','val']:
      if phase=='train':
        model.train()
      else:
        model.eval()

      running_loss = 0.0
      running_corrects = 0.0

      # iterate over data
      for inputs,labels in dataloaders[phase]:
        inputs = inputs.to(device)
        labels = labels.to(device)

        # forward
        # track history if only in train
        with torch.set_grad_enabled(phase=='train'):
          outputs = model(inputs)
          _,preds = torch.max(outputs,1)
          loss = criterion(outputs,labels)

          # backward
          if phase== 'train':
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
        running_loss += loss.item() * inputs.size(0)
        running_corrects += torch.sum(preds == labels.data)
      if phase == 'train':
        scheduler.step()

      epoch_loss = running_loss / dataset_sizes[phase]
      epoch_acc = running_corrects.double() / dataset_sizes[phase]

      print(f'{phase} loss: { epoch_loss:.4f} Acc: {epoch_acc:.4f}')

      # deep copy the model
      if phase == 'val' and epoch_acc > best_acc:
        best_acc = epoch_acc
        best_model_wts = copy.deepcopy(model.state_dict())

    print()

  time_elapsed = time.time()- since
  print(f'training completed in {time_elapsed//60:.0f}m {time_elapsed%60:.0f}s')
  print(f'best val accuracy: {best_acc}')

  #load the best model weights
  model.load_state_dict(best_model_wts)
  return model






In [22]:
model= models.resnet18(pretrained = True)
num_ftrs = model.fc.in_features
model.fc = nn.Linear(num_ftrs, 2)
model.to(device)


/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
  

In [23]:
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(),lr=0.001)
criterion.to(device)

# scheduler that can update the learning rate
step_lr_scheduler = lr_scheduler.StepLR(optimizer, step_size=7, gamma=0.1)

In [24]:
model = train_model(model, criterion, optimizer, step_lr_scheduler, num_epochs=20)

epoch 0/19
----------
train loss: 0.6461 Acc: 0.6016
val loss: 0.5088 Acc: 0.7778

epoch 1/19
----------
train loss: 0.4742 Acc: 0.7886
val loss: 0.3541 Acc: 0.9020

epoch 2/19
----------
train loss: 0.4719 Acc: 0.7724
val loss: 0.2930 Acc: 0.8954

epoch 3/19
----------
train loss: 0.3792 Acc: 0.8537
val loss: 0.2579 Acc: 0.9085

epoch 4/19
----------
train loss: 0.4082 Acc: 0.8374
val loss: 0.2415 Acc: 0.9216

epoch 5/19
----------
train loss: 0.3862 Acc: 0.8252
val loss: 0.2313 Acc: 0.9216

epoch 6/19
----------
train loss: 0.3544 Acc: 0.8374
val loss: 0.2222 Acc: 0.9281

epoch 7/19
----------
train loss: 0.3216 Acc: 0.8618
val loss: 0.2170 Acc: 0.9281

epoch 8/19
----------
train loss: 0.4520 Acc: 0.7724
val loss: 0.2267 Acc: 0.9085

epoch 9/19
----------
train loss: 0.3761 Acc: 0.8374
val loss: 0.2428 Acc: 0.9150

epoch 10/19
----------
train loss: 0.3602 Acc: 0.8496
val loss: 0.2108 Acc: 0.9412

epoch 11/19
----------
train loss: 0.4076 Acc: 0.8455
val loss: 0.2186 Acc: 0.9281

ep